In [51]:
import requests
import bs4
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import time
import re

In [32]:
def scroll_page(browser):
    """
    Automatically control to scroll the webpage to the bottom
    Input: 
        browser: web driver
    Output: None
    """
    
    SCROLL_PAUSE_TIME = 1

    # Get scroll height
    last_height = browser.execute_script("return document.body.scrollHeight")
    while True:
        # Scroll down to bottom
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)
        # Calculate new scroll height and compare with last scroll height
        new_height = browser.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
    return None 

In [20]:
def get_product_links(source):
    """
    Retrive product links from a web source
    Input:
        webpage source
    Output:
        product links as list
    """
    soup = bs4.BeautifulSoup(source)
    domain = "https://products.wholefoodsmarket.com"
    contents = soup.find_all(name="a", class_="ProductCard-Root--3g5WI")
    links = [domain + content.get("href") for content in contents]
    return links

In [33]:
product_links = set()
browser = webdriver.Chrome()
init_url = "https://products.wholefoodsmarket.com/search?sort=relevance"
browser.get(init_url)

no_sub_categ = ["cheese", "seafood"]
buttons = browser.find_elements_by_class_name("LandingPage-SeeAll--2PvpU")
button_nums = len(buttons)

init_action = ActionChains(browser)
for i in range(button_nums):
    init_url = "https://products.wholefoodsmarket.com/search?sort=relevance"
    browser.get(init_url)
    time.sleep(2)
    buttons = browser.find_elements_by_class_name("LandingPage-SeeAll--2PvpU")
    button = buttons[i]
    init_action.click(button).perform()

    categ_url = browser.current_url
    browser.get(categ_url)
    categ = categ_url.split("=")[2]
    if categ not in no_sub_categ and categ != "floral":
        sub_buttons = browser.find_elements_by_class_name("Radio-Label--1a5oe")
        sub_action = ActionChains(browser)
        for sub_button in sub_buttons[1:]:
            sub_action.click(sub_button).perform()
            scroll_page(browser)
            source = browser.page_source
            links = get_product_links(source)
            print("Number of products", len(links))
            product_links.update(links)
            sub_action = ActionChains(browser)
    elif categ == "floral":
        pass
    else:
        scroll_page(browser)
        source = browser.page_source
        links = get_product_links(source)
        print("Number of products", len(links))
        product_links.update(links)

    back_action = ActionChains(browser)
    back_button = browser.find_element_by_class_name("Radio-Label--1a5oe")
    back_action.click(back_button).perform()
    init_action = ActionChains(browser)
browser.close()

https://products.wholefoodsmarket.com/search?sort=relevance&category=produce
produce
Number of products 95
Number of products 14
Number of products 198
https://products.wholefoodsmarket.com/search?sort=relevance&category=dairy-eggs
dairy-eggs
Number of products 52
Number of products 200
Number of products 42
Number of products 134
Number of products 200
https://products.wholefoodsmarket.com/search?sort=relevance&category=floral
floral
https://products.wholefoodsmarket.com/search?sort=relevance&category=breakfast
breakfast
Number of products 157
Number of products 84
https://products.wholefoodsmarket.com/search?sort=relevance&category=cheese
cheese
Number of products 200
https://products.wholefoodsmarket.com/search?sort=relevance&category=frozen-foods
frozen-foods
Number of products 58
Number of products 200
Number of products 103
Number of products 67
Number of products 200
https://products.wholefoodsmarket.com/search?sort=relevance&category=beverages
beverages
Number of products 200
N

In [34]:
print(len(product_links))

5736


In [14]:
# 查找重复项
import collections
print([item for item, count in collections.Counter(product_links).items() if count > 1])

['https://products.wholefoodsmarket.com/product/tomatoes-on-the-vine-0949fc', 'https://products.wholefoodsmarket.com/product/organic-tomatoes-on-the-vine-f63de5', 'https://products.wholefoodsmarket.com/product/sunset-wild-wonders-gourmet-medley-tomatoes-12-oz-106789', 'https://products.wholefoodsmarket.com/product/plantain-fc6ba3', 'https://products.wholefoodsmarket.com/product/whole-foods-market-whole-trade-organic-red-cherry-tomatoes-00a35b', 'https://products.wholefoodsmarket.com/product/tomatoes-on-the-vine-094d4a', 'https://products.wholefoodsmarket.com/product/365-everyday-value-organic-broccoli-florets-4373b5', 'https://products.wholefoodsmarket.com/product/earth-balance-organic-whipped-buttery-spread-f7a020', 'https://products.wholefoodsmarket.com/product/miyokos-creamery-organic-european-style-cultured-vegan-butter-ea06f3', 'https://products.wholefoodsmarket.com/product/earth-balance-vegan-buttery-sticks-0c6091', 'https://products.wholefoodsmarket.com/product/violife-cheddar-s

Find you may also like

In [ ]:
starting_link = product_links[545]
r = requests.get(starting_link)

In [26]:
def get_what_you_may_like(starting_link):
    browser.get(starting_link)
    source = browser.page_source
    soup = bs4.BeautifulSoup(source)
    soups = soup.find_all("a", class_="ProductCard-Root--3g5WI")
    domain = "https://products.wholefoodsmarket.com"
    links = [domain + soup.get("href") for soup in soups]
    return links

In [35]:
you_may_like_links = set()
browser = webdriver.Chrome()
for link in product_links:
    item_links = get_what_you_may_like(link)
    you_may_like_links.update(item_links)
    
browser.close()

In [36]:
len(you_may_like_links)

895

In [39]:
final_links = product_links.union(you_may_like_links)
len(final_links)

6215

In [22]:
product_link = set(product_links)

In [37]:
final_links

{'https://products.wholefoodsmarket.com/product/pom-pomegranate-juice-fc9049',
 'https://products.wholefoodsmarket.com/product/marin-french-cheese-triple-creme-brie-32a894',
 'https://products.wholefoodsmarket.com/product/reese-reese-baby-corn-6fd995',
 'https://products.wholefoodsmarket.com/product/eggology-organic-liquid-egg-whites-6591f9',
 'https://products.wholefoodsmarket.com/product/applegate-organics-the-great-organic-uncured-beef-hot-dog-e3c4e6',
 'https://products.wholefoodsmarket.com/product/cazadores-reposado-tequila-82404d',
 'https://products.wholefoodsmarket.com/product/365-everyday-value-365-everyday-value-vanilla-sandwich-cremes-da5241',
 'https://products.wholefoodsmarket.com/product/late-july-organic-snacks-late-july-organic-snacks-organic-sea-salt-thin-crispy-tortilla-chips-9a1f58',
 'https://products.wholefoodsmarket.com/product/bio-k-plus-mango-fermented-soy-probiotic-12-pk-dda1fc',
 'https://products.wholefoodsmarket.com/product/california-farms-sweetened-condens

In [48]:
with open("final_links.txt", "w") as f:
    for link in final_links:
        f.write(str(link)+"\n")

Retrive information from one item

In [43]:
test_link = list(final_links)[2348]

In [47]:
browser = webdriver.Chrome()
browser.get(test_link)
ingre_nu_buttons = browser.find_elements_by_class_name("Collapsible-Root--3cwwH")
ingre_button = ingre_nu_buttons[0]
ingre_action = ActionChains(browser)
ingre_action.click(ingre_button).perform()

nu_button = ingre_nu_buttons[1]
nu_action = ActionChains(browser)
nu_action.click(nu_button).perform()

source = browser.page_source



In [48]:
browser.close()

In [152]:
soup = bs4.BeautifulSoup(source)

to update: get serving size and ingredients using re.

In [208]:
def get_prod_info(browser, link):
    browser.get(link)
    
    try:
        ing_nu_buttons = browser.find_elements_by_class_name("Collapsible-Root--3cwwH")

        ing_button = ing_nu_buttons[0]
        ing_action = ActionChains(browser)
        ing_action.click(ing_button).perform()

        nu_button = ing_nu_buttons[1]
        nu_action = ActionChains(browser)
        nu_action.click(nu_button).perform()
    except Exception:
        print("No ingredients and No nutrition!")

    try:
        detail_button = browser.find_element_by_class_name("Collapsible-Title--30gLK disable-click-focus")
        detail_action = ActionChains(browser)
        detail_action.click(detail_button).perform()
    except Exception:
        print("No product details!")

    source = browser.page_source
    soup = bs4.BeautifulSoup(source)
    
    prod_dic = dict()
    try:
        brand_nm = soup.find("div", class_="ProductHeader-Brand--1Yx6l").get_text()
        prod_dic["brand_nm"] = brand_nm.lower()
    except Exception:
        print("No brand name!")
        prod_dic["brand_nm"] = "Whole Foods".lower()
    
    prod_nm = soup.find("h1", class_="ProductHeader-Name--1ysBV").get_text()
    prod_dic["prod_nm"] = prod_nm.lower()
    
    try:
        ingrds = soup.find("div", class_="Product-SecondaryText--wF9l_").get_text()
        prod_dic["ingredients"] = ingrds.lower()
    except Exception:
        print("No ingredients!")
        
    try:
        ingrds = soup.find("div", class_="Product-CollapsibleStatement--1VluS").get_text()
        prod_dic["ingredients"] = ingrds.lower()
    except Exception:
        print("No ingredients info from product details!")
    
    try:
        allgrgs = soup.find("div", class_="Product-CollapsibleStatement--1VluS").get_text()
        prod_dic["allergens"] = allgrgs.lower()
    except Exception:
        print("No allgergens!")
    
    try:
        contents = soup.find_all("div", class_="Row__15gM6")
        nutri_dict = dict()
        for line in contents[1:]:
            line = line.get_text()
            try:
                nutri = re.search(r"([A-Za-z]+\s?[A-Za-z]*)", line).group(1)
                quant = line.lstrip(nutri).strip()
                if len(quant) != 0:
                    nutri_dict[nutri.lower()] = quant.lower()
            except Exception:
                pass
        prod_dic["nutrition"] = nutri_dict
    except Exception:
        print("No ingredients table!")

    labels = soup.find_all("div", class_="Diets-DietName--1T3K1")
    prod_dic["labels"] = set()
    for label in labels:
        prod_dic["labels"].add(label.get_text())

    return prod_dic

In [ ]:
prod_link = list(final_links)

In [214]:
browser = webdriver.Chrome()
prods = []
idx = 0
for i in prod_link[118:]:
    print(get_prod_info(browser, i))
    prods.append(get_prod_info(browser, i))
    print(cnt, "-" * 50)
    cnt += 1
browser.close()

No product details!
No brand name!
No ingredients info from product details!
No allgergens!
{'brand_nm': 'whole foods', 'prod_nm': 'organic plum (roma) tomatoes, 1 tomato', 'ingredients': 'organic plum roma tomato', 'nutrition': {'serving size': '1 tomato (62 g)', 'calories': '11', 'total fat': '0.12g', 'saturated fat': '0.02g', 'trans fat': '0g', 'polyunsaturated fat': '0.05g', 'monounsaturated fat': '0.02g', 'cholesterol': '0mg', 'sodium': '3.1mg', 'total carbohydrates': '2.4g', 'dietary fiber': '0.74g', 'sugars': '1.63g', 'protein': '0.55g', 'vitamin d': '0iu', 'calcium': '6.2mg', 'iron': '0.17mg', 'potassium': '147mg', 'vitamin a': '516iu', 'vitamin c': '8mg', 'vitamin e': '0.5iu', 'vitamin k': '4.9µg', 'thiamin': '0.02mg', 'riboflavin': '0.01mg', 'niacin': '0.37mg', 'vitamin b': '120µg', 'folate': '9µg', 'pantothenic acid': '0.06mg', 'phosphorus': '15mg', 'magnesium': '7mg', 'zinc': '0.11mg', 'selenium': '0µg', 'copper': '0.04mg', 'manganese': '0.07mg'}, 'labels': {'Paleo Friendly

No product details!
No ingredients info from product details!
No allgergens!
129 --------------------------------------------------
No product details!
{'brand_nm': '365 everyday value®', 'prod_nm': 'frosted wheat squares cereal (bite-sized), 16 oz', 'ingredients': 'contains wheat (whole wheat)', 'allergens': 'contains wheat (whole wheat)', 'nutrition': {'serving size': '1 cup (60 g)', 'calories': '230', 'total fat': '1g', 'saturated fat': '0g', 'trans fat': '0g', 'cholesterol': '0mg', 'sodium': '5mg', 'total carbohydrates': '50g', 'dietary fiber': '6g', 'sugars': '12g', 'includes added': 'sugar12g', 'protein': '6g', 'vitamin d': '0mcg', 'calcium': '0mg', 'iron': '18mg', 'potassium': '190mg', 'thiamin': '0mg', 'riboflavin': '0mg', 'niacin': '5mg', 'vitamin b': '121mcg', 'folate': '185mcg', 'everyday value': '365 everyday value®'}, 'labels': {'Dairy Free', 'Vegan', 'Low Fat', 'Low Sodium', 'Vegetarian'}}
No product details!
130 --------------------------------------------------
No produ

No product details!
138 --------------------------------------------------
No product details!
No ingredients info from product details!
No allgergens!
{'brand_nm': 'butternut mountain farm: the vermont maple sugar company', 'prod_nm': '100% pure vermont maple syrup - grade a dark color (8 oz)', 'ingredients': 'pure maple syrup', 'nutrition': {'serving size': '4 tbsp (60 ml)', 'calories': '200', 'total fat': '0g', 'saturated fat': '0g', 'trans fat': '0g', 'cholesterol': '0mg', 'sodium': '5mg', 'total carbohydrates': '53g', 'dietary fiber': '0g', 'sugars': '53g', 'protein': '0g', 'everyday value': '365 everyday value®'}, 'labels': {'Paleo Friendly', 'Kosher+', 'Low Fat*', 'Dairy Free', 'Vegan', 'Low Sodium**', 'Vegetarian'}}
No product details!
No ingredients info from product details!
No allgergens!
139 --------------------------------------------------
No ingredients and No nutrition!
No product details!
No ingredients!
No ingredients info from product details!
No allgergens!
{'brand_

No product details!
149 --------------------------------------------------
No product details!
{'brand_nm': 'california farms', 'prod_nm': 'evaporated milk organic, 12 fl. oz.', 'ingredients': 'contains milk and dairy (organic milk)', 'allergens': 'contains milk and dairy (organic milk)', 'nutrition': {'serving size': '2 tbsp (30 ml)', 'calories': '40', 'total fat': '2g', 'saturated fat': '1.5g', 'trans fat': '0g', 'cholesterol': '10mg', 'sodium': '30mg', 'total carbohydrates': '3g', 'sugars': '3g', 'protein': '2g', 'potassium': '90mg', 'everyday value': '365 everyday value®'}, 'labels': {'Kosher+', 'Low Sodium', 'Sugar Conscious', 'Whole Foods Diet', 'Vegetarian'}}
No product details!
150 --------------------------------------------------
No product details!
{'brand_nm': "bob's red mill", 'prod_nm': 'organic thick rolled oats, 32 oz', 'ingredients': 'produced in a facility that also processes soy, tree nuts', 'allergens': 'produced in a facility that also processes soy, tree nuts', 'n

No product details!
No ingredients info from product details!
No allgergens!
159 --------------------------------------------------
No product details!
{'brand_nm': '365 everyday value®', 'prod_nm': '365 everyday value® organic mini pretzel twists (unsalted), 8 oz', 'ingredients': 'contains soy (organic expeller pressed soybean oil), wheat (organic wheat flour)', 'allergens': 'contains soy (organic expeller pressed soybean oil), wheat (organic wheat flour)', 'nutrition': {'serving size': '1 oz (28 g)', 'calories': '110', 'total fat': '0.5g', 'saturated fat': '0g', 'trans fat': '0g', 'polyunsaturated fat': '0g', 'monounsaturated fat': '0g', 'cholesterol': '0mg', 'sodium': '40mg', 'total carbohydrates': '23g', 'dietary fiber': '2g', 'sugars': '< 1g', 'includes added': 'sugar0g', 'protein': '3g', 'vitamin d': '0mcg', 'calcium': '0mg', 'iron': '0.4mg', 'potassium': '0mg', 'everyday value': '365 everyday value®'}, 'labels': {'Kosher+', 'Dairy Free', 'Organic', 'Vegan', 'Low Fat', 'Low Sodiu

No product details!
170 --------------------------------------------------
No product details!
No ingredients info from product details!
No allgergens!
{'brand_nm': '365 everyday value®', 'prod_nm': 'organic frozen blueberries, 32 oz', 'ingredients': 'organic blueberries.', 'nutrition': {'serving size': '1 cup (140 g)', 'calories': '70', 'total fat': '1g', 'saturated fat': '0g', 'trans fat': '0g', 'cholesterol': '0mg', 'sodium': '0mg', 'total carbohydrates': '17g', 'dietary fiber': '4g', 'sugars': '12g', 'includes added': 'sugar0g', 'protein': '1g', 'vitamin d': '0mcg', 'calcium': '0mg', 'iron': '0mg', 'potassium': '0mg', 'everyday value': '365 everyday value®'}, 'labels': {'Paleo Friendly', 'Kosher+', 'Low Fat*', 'Whole Foods Diet', 'Dairy Free', 'Organic', 'Vegan', 'Engine 2', 'Keto Friendly', 'Low Sodium**', 'Vegetarian'}}
No product details!
No ingredients info from product details!
No allgergens!
171 --------------------------------------------------
No product details!
{'brand_nm

No product details!
182 --------------------------------------------------
No ingredients and No nutrition!
No product details!
No ingredients!
No ingredients info from product details!
No allgergens!
{'brand_nm': 'ballast point', 'prod_nm': 'ballast point sculpin ipa cans (6 pk), 12 oz cans', 'nutrition': {}, 'labels': set()}
No ingredients and No nutrition!
No product details!
No ingredients!
No ingredients info from product details!
No allgergens!
183 --------------------------------------------------
No product details!
{'brand_nm': '365 everyday value®', 'prod_nm': 'block cheese packed in brine feta cheese, 16 oz', 'ingredients': 'contains milk and dairy (cultured pasteurized milk)', 'allergens': 'contains milk and dairy (cultured pasteurized milk)', 'nutrition': {'serving size': '1 oz (28 g)', 'calories': '60', 'total fat': '4g', 'saturated fat': '2g', 'trans fat': '0g', 'cholesterol': '15mg', 'sodium': '370mg', 'total carbohydrates': '1g', 'dietary fiber': '0g', 'sugars': '0g', 

No product details!
{'brand_nm': 'sierra nevada cheese', 'prod_nm': 'organic vat cultured butter, 8 oz', 'ingredients': 'contains milk and dairy (certified organic pasteurized cream)', 'allergens': 'contains milk and dairy (certified organic pasteurized cream)', 'nutrition': {'serving size': '1 tbsp (14 g)', 'calories': '110', 'total fat': '12g', 'saturated fat': '7g', 'trans fat': '0g', 'cholesterol': '30mg', 'sodium': '20mg', 'total carbohydrates': '0g', 'protein': '0g'}, 'labels': {'Low Sodium*', 'Keto Friendly', 'Vegetarian'}}
No product details!
194 --------------------------------------------------
No product details!
{'brand_nm': '365 everyday value®', 'prod_nm': 'organic triple berry instant oatmeal, 2 oz', 'ingredients': 'produced in a facility that also processes milk and dairy, peanuts, soy, tree nuts, wheat', 'allergens': 'produced in a facility that also processes milk and dairy, peanuts, soy, tree nuts, wheat', 'nutrition': {'serving size': '1 cup (57 g)', 'calories': '22

No product details!
{'brand_nm': 'california farms', 'prod_nm': 'evaporated milk, 12 fl oz', 'ingredients': 'contains milk and dairy (milk)', 'allergens': 'contains milk and dairy (milk)', 'nutrition': {'serving size': '2 tbsp (30 ml)', 'calories': '40', 'total fat': '2g', 'saturated fat': '1g', 'trans fat': '0g', 'cholesterol': '5mg', 'sodium': '25mg', 'total carbohydrates': '3g', 'sugars': '3g', 'protein': '2g', 'potassium': '90mg', 'nature': "'s charm", 'everyday value': '365 everyday value®'}, 'labels': {'Kosher+', 'Low Sodium', 'Sugar Conscious', 'Whole Foods Diet', 'Vegetarian'}}
No product details!
206 --------------------------------------------------
No product details!
{'brand_nm': "dr. praeger's sensible foods", 'prod_nm': 'spinach littles, 10 oz', 'ingredients': 'contains eggs (egg whites)', 'allergens': 'contains eggs (egg whites)', 'nutrition': {'serving size': '5 piece (85 g)', 'calories': '140', 'total fat': '8g', 'saturated fat': '0.5g', 'trans fat': '0g', 'cholesterol

AttributeError: 'NoneType' object has no attribute 'get_text'

In [213]:
part1 = prods
len(part1)

120

In [217]:
part2 = prods
len(part2)

92